<a href="https://colab.research.google.com/github/Kira1108/pretrain-learn/blob/main/pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!free -g

               total        used        free      shared  buff/cache   available
Mem:              50           0          46           0           4          49
Swap:              0           0           0


In [3]:
from IPython.display import clear_output
!pip install datasets
!pip install fasttext
clear_output()

In [4]:
import warnings
warnings.filterwarnings("ignore")

import datasets
pretraining_dataset = datasets.load_dataset(
    "upstage/Pretraining_Dataset",
    split="train"
)

pretraining_dataset.parquet:   0%|          | 0.00/150M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [5]:
from pprint import pprint

pretraining_dataset = pretraining_dataset.select_columns(
    ["text"]
)

pprint(pretraining_dataset[0],width = 160)

{'text': 'In 1793 Zaman Shah, a grandson of Ahmad Shah Durrani, won a brief war of succession to become ruler of Afghanistan. The support of Painda Khan, '
         'chief of the Baraksai branch of the Durrani tribe, was decisive in his victory. In the next fifty year., the brothers of Zaman shah and the sons of '
         'Painda Khan were to dominate the affairs of Afghanistan. The Durrani tribe was very large with several branches and numerous clans. 1 Abmad Shah and '
         'his successors belonged to the Sadozai clan, but other clans, such as the Mohammedzai of Painda Khan, were larger and more powerful and this '
         'situation caused many problems.\n'
         "Mahmud had revolted unsuccessfully several times with Persian backing, but now with Fateh Khan's help he was able to defeat Zaman who was captured "
         "and blinded. Mahmud's position was insecure however. Persian invasions threatened, the tribes were discontented, and another brother of Zaman, "
         "S

In [7]:
instruction_dataset = datasets.load_dataset(
    "c-s-ale/alpaca-gpt4-data",
    split='train'
)
print(instruction_dataset)
print("\nInstruction Dataset Overview:")
pprint(instruction_dataset[0])

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 52002
})

Instruction Dataset Overview:
{'input': '',
 'instruction': 'Give three tips for staying healthy.',
 'output': '1. Eat a balanced and nutritious diet: Make sure your meals are '
           'inclusive of a variety of fruits and vegetables, lean protein, '
           'whole grains, and healthy fats. This helps to provide your body '
           'with the essential nutrients to function at its best and can help '
           'prevent chronic diseases.\n'
           '\n'
           '2. Engage in regular physical activity: Exercise is crucial for '
           'maintaining strong bones, muscles, and cardiovascular health. Aim '
           'for at least 150 minutes of moderate aerobic exercise or 75 '
           'minutes of vigorous exercise each week.\n'
           '\n'
           '3. Get enough sleep: Getting enough quality sleep is crucial for '
           'physical and mental well-being. It helps to regulate

In [8]:
# Import some required packages
import os
import requests

def download_code(local_dir:str = "./code"):
    os.makedirs(local_dir, exist_ok=True)

    urls = [
        "https://raw.githubusercontent.com/TheAlgorithms/Python/master/searches/double_linear_search_recursion.py",
        "https://raw.githubusercontent.com/KosingZhu/tensorflow/master/tensorflow/python/tools/module_util.py",
        "https://raw.githubusercontent.com/EricRemmerswaal/tensorflow/master/tensorflow/python/distribute/distribute_coordinator_context.py",
        "https://raw.githubusercontent.com/computationalartist/tensorflow/master/tensorflow/python/ops/numpy_ops/integration_test/benchmarks/numpy_mlp.py",
        "https://raw.githubusercontent.com/Van-an/tensorflow/master/tensorflow/python/distribute/coordinator/values.py",
        "https://raw.githubusercontent.com/nkgwer/tensorflow/master/tensorflow/lite/tools/visualize.py",
        "https://raw.githubusercontent.com/gitblazer/youtube-dl/master/youtube_dl/version.py",
        "https://raw.githubusercontent.com/Joshua-Barawa/My-Photos/master/venv/lib/python3.8/site-packages/django/contrib/messages/__init__.py",
        "https://raw.githubusercontent.com/PaliC/pytorch/master/test/fx/test_subgraph_rewriter.py"
    ]

    for url in urls:
        print(f"Working on url: {url}")
        response = requests.get(url)
        file_name = os.path.basename(url)
        file_path = os.path.join(local_dir, file_name)

        if response.status_code == 200:
            with open(file_path, "wb") as file:
                file.write(response.content)

def create_dataset(local_dir:str):

    code_dataset = []
    for file in os.listdir(code_dir):
        if not file.startswith("."):
            code_dataset.append(
                {'text': open(os.path.join(code_dir, file), 'r').read()}
            )


    # 从数组创建huggingface dataset
    return datasets.Dataset.from_list(code_dataset)

code_dir = "./code"
download_code(code_dir)
code_dataset = create_dataset(code_dir)

Working on url: https://raw.githubusercontent.com/TheAlgorithms/Python/master/searches/double_linear_search_recursion.py
Working on url: https://raw.githubusercontent.com/KosingZhu/tensorflow/master/tensorflow/python/tools/module_util.py
Working on url: https://raw.githubusercontent.com/EricRemmerswaal/tensorflow/master/tensorflow/python/distribute/distribute_coordinator_context.py
Working on url: https://raw.githubusercontent.com/computationalartist/tensorflow/master/tensorflow/python/ops/numpy_ops/integration_test/benchmarks/numpy_mlp.py
Working on url: https://raw.githubusercontent.com/Van-an/tensorflow/master/tensorflow/python/distribute/coordinator/values.py
Working on url: https://raw.githubusercontent.com/nkgwer/tensorflow/master/tensorflow/lite/tools/visualize.py
Working on url: https://raw.githubusercontent.com/gitblazer/youtube-dl/master/youtube_dl/version.py
Working on url: https://raw.githubusercontent.com/Joshua-Barawa/My-Photos/master/venv/lib/python3.8/site-packages/djan

In [9]:
dataset = datasets.concatenate_datasets(
    [pretraining_dataset, code_dataset]
)
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 60007
})


In [12]:
import heapq

def paragraph_length_filter(x):
    """Returns False iff a page has too few lines or lines are too short."""
    lines = x['text'].split('\n')
    if (
        len(lines) < 3
        or min(heapq.nlargest(3, [len(line) for line in lines])) < 3
    ):
        return False
    return True

dataset = dataset.filter(
    paragraph_length_filter,
    load_from_cache_file=False
)

Filter:   0%|          | 0/60007 [00:00<?, ? examples/s]

In [13]:
dataset.num_rows

52356

In [15]:
def find_duplicates(paragraphs):
    """
    Use this function to find the number of repetitions
    in the paragraphs.
    """
    unique_x = set()
    duplicate_chars = 0
    duplicate_elements = 0
    for element in paragraphs:
        if element in unique_x:
            duplicate_chars += len(element)
            duplicate_elements += 1
        else:
            unique_x.add(element)
    return duplicate_elements, duplicate_chars

import re

def paragraph_repetition_filter(x):
    """
    Returns False iff a page has too many repetitions.
    """
    text = x['text']
    paragraphs = re.compile(r"\n{2,}").split(text.strip())                # Split by paragraphs (2 or more newlines)
    paragraphs_duplicates, char_duplicates = find_duplicates(paragraphs)  # Find number of duplicates in paragraphs
    if paragraphs_duplicates / len(paragraphs) > 0.3:
        return False
    if char_duplicates / len(text) > 0.2:
        return False
    return True

In [16]:
dataset = dataset.filter(
    paragraph_repetition_filter,
    load_from_cache_file=False
)

Filter:   0%|          | 0/52356 [00:00<?, ? examples/s]

In [17]:
dataset.num_rows

52326

In [18]:
def deduplication(ds):
    def dedup_func(x):
        """Use this function to remove duplicate entries"""
        if x['text'] in unique_text:
            return False
        else:
            unique_text.add(x['text'])
            return True

    unique_text = set()

    ds = ds.filter(dedup_func, load_from_cache_file=False, num_proc=1)
    return ds

dataset = deduplication(dataset)

Filter:   0%|          | 0/52326 [00:00<?, ? examples/s]

In [19]:
dataset.num_rows

43597

In [24]:
import urllib
from fasttext.FastText import _FastText

def english_language_filter(ds):
    # load language detection model
    model = _FastText('/content/drive/MyDrive/experiments/L2_language_model.bin')

    def is_english(x):
        # Predict language of the text and probability
        language, score = model.predict(x['text'].replace("\n", ""))

        language = language[0].split("__")[2]
        return score > 0.4 and language == "en" # change code here if building a model in another language

    ds = ds.filter(is_english, load_from_cache_file=False, num_proc=1)
    return ds

dataset = english_language_filter(dataset)

Parameter 'function'=<function english_language_filter.<locals>.is_english at 0x7edfc0164900> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Filter:   0%|          | 0/43597 [00:00<?, ? examples/s]

In [28]:
file_path = "/content/drive/MyDrive/experiments/preprocessed_dataset.parquet"
dataset.to_parquet(file_path)

Creating parquet from Arrow format:   0%|          | 0/41 [00:00<?, ?ba/s]

197100832

In [27]:
# import pandas as pd

# pd.read_parquet("/content/data/preprocessed_dataset.parquet")